In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os

# Check that the image file exist in currenty dir
current_dir = os.getcwd()
image_filename = 'img_cow_mountain.jpg'

checkFile = current_dir + '\\' + image_filename 
 
if not os.path.isfile(checkFile):
    raise ValueError(f'File {image_filename} does not exist in current directory')
    
# Read and show the image
image = plt.imread(image_filename, format="jpg")

plt.figure()
plt.imshow(image)
plt.show()

print(f'image array dimensions={image.shape}')


In [ ]:
from scipy import ndimage

# Rotate the image
plt.figure()
image_rotated = ndimage.rotate(image, -90)
plt.imshow(image_rotated)
plt.show()


In [ ]:
# Show image in black & white

from PIL import Image

image_gray = Image.open(image_filename).convert('L')
image_gray_rotated = ndimage.rotate(image_gray, -90)
image_gray_rotated_array=np.asarray(image_gray_rotated)

plt.figure()
plt.imshow(image_gray_rotated_array,cmap='gray', vmin = 0, vmax = 255)
plt.show()

print(f'image (GRAY) numpy array dimensions={image.shape}')


In [ ]:
# Show image in fancy colors
def color_fancy(image):
    cmaps = ['afmhot', 'autumn', 'bone', 'binary', 'bwr', 'brg', 
             'CMRmap', 'cool', 'copper', 'cubehelix', 'Greens', 'winter']
    fig, ax = plt.subplots(4, 3, figsize=(15,20), sharey = True)
    for index, cmap_name in enumerate(cmaps):
        index_column = index % 3
        index_row = index // 3
        ax[index_row][index_column].imshow(image, cmap = cmap_name)
        ax[index_row][index_column].set_title(cmap_name, fontsize = 15)

color_fancy(image_gray_rotated_array)


In [ ]:
# Split the RGB channels

def rgb_splitter(image):
    rgb_list = ['Reds','Greens','Blues']
    fig, ax = plt.subplots(1, 3, figsize=(15,5), sharey = True)
    for index, rgb_name in enumerate(rgb_list):
        ax[index].imshow(image[:,:,index], cmap = rgb_name)
        ax[index].set_title(rgb_name, fontsize = 15)

rgb_splitter(image_rotated)

In [ ]:
# Identify colored objects from the RGB channels

def rgb_filter(image, filter_color, red, green, blue):

    if (filter_color == 'red'):
        image_filtered = (image[:,:,0] > red) & (image[:,:,1] < green) & (image[:,:,2] < blue)
    elif (filter_color == 'green'):
        image_filtered = (image[:,:,0] < red) & (image[:,:,1] > green) & (image[:,:,2] < blue)
    elif (filter_color == 'blue'):
        image_filtered = (image[:,:,0] < red) & (image[:,:,1] < green) & (image[:,:,2] > blue)
    else:
        raise ValueError(f'filter_color={filter_color} must be one of "red", "green", "blue"')
    
    image_new = image.copy()
    image_new[:, :, 0] = image_new[:, :, 0] * image_filtered
    image_new[:, :, 1] = image_new[:, :, 1] * image_filtered
    image_new[:, :, 2] = image_new[:, :, 2] * image_filtered
    
    return image_new

image_rotated_red = rgb_filter(image_rotated, filter_color='red', red=80, green=100, blue=100)
image_rotated_green = rgb_filter(image_rotated, filter_color='green', red=150, green=100, blue=100)
image_rotated_blue = rgb_filter(image_rotated, filter_color='blue', red=100, green=150, blue=50)

fig, ax = plt.subplots(1, 3, figsize=(15,5), sharey = True)
    
ax[0].imshow(image_rotated_red)
ax[0].set_title('Red filtered', fontsize = 15)
ax[1].imshow(image_rotated_green)
ax[1].set_title('Green filtered', fontsize = 15)
ax[2].imshow(image_rotated_blue)
ax[2].set_title('Blue filtered', fontsize = 15)

plt.show()


In [ ]:
# Split image in HSV (Hue Saturation Value)

import matplotlib.colors as mcolors

def display_as_hsv(image):
    
    img_hsv = mcolors.rgb_to_hsv(image)
    
    hsv_list = ['Hue','Saturation','Value']
    fig, ax = plt.subplots(1, 3, figsize=(15,5), sharey = True)
    
    im0 = ax[0].imshow(img_hsv[:,:,0], cmap = 'hsv')
    ax[0].set_title(hsv_list[0], fontsize = 15)
    fig.colorbar(im0, ax=ax[0])

    im1 = ax[1].imshow(img_hsv[:,:,1], cmap = 'Greys')
    ax[1].set_title(hsv_list[1], fontsize = 15)
    fig.colorbar(im1, ax=ax[1])
    
    im2 = ax[2].imshow(img_hsv[:,:,2], cmap = 'gray')
    ax[2].set_title(hsv_list[2], fontsize = 15)
    fig.colorbar(im2, ax=ax[2])

display_as_hsv(image_rotated)

In [ ]:
# Identify colored objects from the HSV channels

def create_hsv_mask(img_hsv, mask_type, lower, upper):
    if (mask_type == 'h'):
        lower_mask = img_hsv [:,:,0] > lower
        upper_mask = img_hsv [:,:,0] < upper
    elif (mask_type == 's'):
        lower_mask = img_hsv [:,:,1] > lower
        upper_mask = img_hsv [:,:,1] < upper
    elif (mask_type == 'v'):
        lower_mask = img_hsv [:,:,2] > lower
        upper_mask = img_hsv [:,:,2] < upper
    else:
        raise ValueError(f'mask_type={mask_type} must be one of "h", "s", "v"')
        
    return upper_mask*lower_mask

def mask_image(img_rgb, mask):
    red = img_rgb[:,:,0]*mask
    green = img_rgb[:,:,1]*mask
    blue = img_rgb[:,:,2]*mask
    return np.dstack((red,green,blue))


image_hsv = mcolors.rgb_to_hsv(image_rotated)

# Masks for Cow
mask_cow_h = create_hsv_mask(image_hsv, 'h', 0.00, 0.08)
mask_cow_s = create_hsv_mask(image_hsv, 's', 0.40, 1.00)
mask_cow_final = mask_cow_h*mask_cow_s

# Masks for Gras
mask_gras_h = create_hsv_mask(image_hsv, 'h', 0.18, 0.40)
mask_gras_s = create_hsv_mask(image_hsv, 's', 0.30, 0.50)
mask_gras_final = mask_gras_h*mask_gras_s

# Masks for sky
mask_sky_h = create_hsv_mask(image_hsv, 'h', 0.40, 0.80)
mask_sky_s = create_hsv_mask(image_hsv, 's', 0.40, 1.00)
mask_sky_final = mask_sky_h*mask_sky_s

# Images filtered
image_cow_filtered = mask_image(image_rotated, mask_cow_final)
image_sky_filtered = mask_image(image_rotated, mask_sky_final)
image_gras_filtered = mask_image(image_rotated, mask_gras_final)


fig, ax = plt.subplots(1, 3, figsize=(15,5), sharey = True)

ax[0].imshow(image_cow_filtered);
ax[0].set_title('Cow filtered', fontsize = 15)
ax[1].imshow(image_gras_filtered);
ax[1].set_title('Gras filtered', fontsize = 15)
ax[2].imshow(image_sky_filtered);
ax[2].set_title('Sky filtered', fontsize = 15)

plt.plot()
    